In [ ]:
## import the postgres connector
import psycopg2

## INSERT THE PASSWORD

## establishing the connection
conn = psycopg2.connect(
   database="test_db", user='shoc', password='JustKeepSwimming', host='pg_container', port= '5432'
)

## Creating a cursor object using the cursor() method
cursor = conn.cursor()

cursor = conn.cursor()

## Executing an SQL function using the execute() method
cursor.execute("select * from loki_aggregated")

## Fetch a single row using fetchone() method.
data = cursor.fetchall()
print("Pulled all of the data in loki_aggregated: ", type(data), " of length ", len(data))

## Closing the connection
conn.close()

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
from re import search
import matplotlib.pyplot as plt

## Make a dataframe from data
df = pd.DataFrame(data)

## For some reason when we pull from postgres we don't get column
##     names, so go ahead and add them back
df.columns =['container id', 'role', 'user name', 'process', 'message start', 'message stop', 'start time', 'stop time']

## I wrote a custom function to convert the cell to a datetime object
def convertStringToDateTime(cell):
    tmp = cell.to_string(header=False, index=False)
    if tmp == 'NaT':
        return None
    
    try:
        return datetime.strptime(tmp, '%Y-%m-%d %H:%M:%S.%f')
    except ValueError:
        return datetime.strptime(tmp, '%Y-%m-%d %H:%M:%S')


    
## Add another column for our df that calculates the diffence 
##     between the start and stop time
timeDelta = []

## We also need to convert the process and role into ints
processValue = []
roleValue = []

for i in range(0, len(df.index)):
    start = convertStringToDateTime(df.loc[i, ['start time']])
    stop = convertStringToDateTime(df.loc[i, ['stop time']])
    
    if start == None or stop == None:
        timeDelta.append(None)
    else:
        timeDelta.append((stop - start).total_seconds() / 60)
    
    processValue.append(int(df.loc[i, ['process']].to_string(header=False, index=False)[7:]))
    roleValue.append(int(df.loc[i, ['role']].to_string(header=False, index=False)[4:]))

## Now we are done add the time take column
df['delta'] = timeDelta
df['processValue'] = processValue
df['roleValue'] = roleValue

## Add a heat map of our data
pearson_corr = df.corr(method='pearson')[['delta',
                                         'processValue',
                                         'roleValue']]

fig, ax = plt.subplots(figsize=(10, 10))
ax.matshow(pearson_corr)

plt.yticks(range(0, len(pearson_corr.index)),
           pearson_corr.index, fontsize=6)

plt.xticks(range(0, len(pearson_corr.columns)),
           pearson_corr.columns, fontsize=6, rotation=90)

plt.show()